In [1]:
import torch
from pytorch_quantization import tensor_quant

from torch import nn, outer

from pytorch_quantization import tensor_quant
import pytorch_quantization.nn as quant_nn
import pytorch_quantization
pytorch_quantization.__version__
from tqdm import tqdm


In [2]:
from datasets.yolo import YOLO
from engine.detector import Detector
from model.squeezedet import SqueezeDet, SqueezeDetWithLoss
from utils.config import Config
from utils.model import load_model
from PIL import Image
import numpy as np
import argparse
import os
from utils.misc import init_env

In [3]:
args = {'_': '_'}
cfg = Config().parse(args)

init_env(cfg) 
 
cfg.mode = 'eval'
cfg.exp_id = "test_lpd_trt_calib"
cfg.dataset = 'lpr'
cfg.load_model = '../exp/exp14_128_mobv2_stride8/model_best.pth' 
cfg.batch_size = 1
cfg.arch = 'mobilenet_v2'
cfg.num_workers = 0
cfg.save_intervals = 1
cfg.stride = 8
cfg.debug = 2

def update_exp_dir(cfg, exp_id):
    cfg.save_dir = os.path.join(cfg.exp_dir, exp_id)
    os.makedirs(cfg.save_dir, exist_ok=True)
    cfg.log_file = os.path.join(cfg.save_dir, 'training_logs.txt')
    os.remove(cfg.log_file) if os.path.exists(cfg.log_file) else None
    cfg.debug_dir = os.path.join(cfg.save_dir, 'debug')

update_exp_dir(cfg, cfg.exp_id)

Config().print(cfg)


arch                           mobilenet_v2
batch_size                     1
bbox_loss_weight               20.0
chunk_sizes                    [32]
class_loss_weight              1.0
data_dir                       /media/gj_data/QAT_Squeezedet/SqueezeDet-PyTorch/data
dataset                        lpr
debug                          2
debug_dir                      /media/gj_data/QAT_Squeezedet/SqueezeDet-PyTorch/exp/test_lpd_trt_calib/debug
device                         cuda
drift_prob                     1.0
dropout_prob                   0
exp_dir                        /media/gj_data/QAT_Squeezedet/SqueezeDet-PyTorch/exp
exp_id                         test_lpd_trt_calib
flip_prob                      0.5
forbid_resize                  False
gpus                           [0]
gpus_str                       0
grad_norm                      0.5
keep_top_k                     64
load_model                     ../exp/exp14_128_mobv2_stride8/model_best.pth
load_pretrained               

In [213]:
# from eval import eval
# eval(cfg)

In [4]:
from engine.detector import Detector
from model.squeezedet import SqueezeDetWithLoss
from utils.model import load_model
from utils.misc import load_dataset



In [5]:
dataset = load_dataset(cfg.dataset)('val', cfg)
cfg = Config().update_dataset_info(cfg, dataset)
Config().print(cfg)


anchors                        [[  4.   4.   3.   3.]
 [  4.   4.   6.   5.]
 [  4.   4.   9.   5.]
 ...
 [124. 124.  10.   9.]
 [124. 124.  10.  12.]
 [124. 124.  15.   8.]]
anchors_per_grid               6
arch                           mobilenet_v2
batch_size                     1
bbox_loss_weight               20.0
chunk_sizes                    [32]
class_loss_weight              1.0
class_names                    0
data_dir                       /media/gj_data/QAT_Squeezedet/SqueezeDet-PyTorch/data
dataset                        lpr
debug                          2
debug_dir                      /media/gj_data/QAT_Squeezedet/SqueezeDet-PyTorch/exp/test_lpd_trt_calib/debug
device                         cuda
drift_prob                     1.0
dropout_prob                   0
exp_dir                        /media/gj_data/QAT_Squeezedet/SqueezeDet-PyTorch/exp
exp_id                         test_lpd_trt_calib
flip_prob                      0.5
forbid_resize                  False
gpu

In [9]:
print (dataset.phase)

val


In [6]:
if cfg.mode=='eval':
        model = SqueezeDetWithLoss(cfg)
        model = load_model(model, cfg.load_model, cfg)


detect = model.detect
model.detect = True
detector = Detector(model, cfg)
print(detector.model)

loaded model ../exp/exp14_128_mobv2_stride8/model_best.pth, epoch 410
Model successfully loaded.
SqueezeDetWithLoss(
  (base): SqueezeDetBase(
    (quant): QuantStub()
    (dequant): DeQuantStub()
    (features): Sequential(
      (0): ConvBNReLU(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): ConvBNReLU(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU()
          )
          (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (skip_add): FloatFunctional(

In [218]:
from pytorch_quantization import nn as quant_nn
from pytorch_quantization import calib
from pytorch_quantization.tensor_quant import QuantDescriptor

from absl import logging
logging.set_verbosity(logging.INFO)  # Disable logging as they are too noisy in notebook

In [219]:
quant_desc_input = QuantDescriptor(calib_method='histogram')
quant_nn.QuantConv2d.set_default_quant_desc_input(quant_desc_input)
quant_nn.QuantLinear.set_default_quant_desc_input(quant_desc_input)

In [220]:
from pytorch_quantization import quant_modules
quant_modules.initialize()

### Calibration Functions

In [221]:
def collect_stats(model, data_loader, num_batches):
    """Feed data to the network and collect statistic"""

    # Enable calibrators
    for name, module in model.named_modules():
        if isinstance(module, quant_nn.TensorQuantizer):
            if module._calibrator is not None:
                module.disable_quant()
                module.enable_calib()
            else:
                module.disable()

    # for i, (image, _) in tqdm(enumerate(data_loader), total=num_batches):
    #     model(image.cuda())
    #     if i >= num_batches:
    #         break
    # since lpr dataloader return a dict, so writing it in this way
    for i, (batch) in tqdm(enumerate(data_loader), total=num_batches):
        for k in batch:
                if 'image_meta' not in k:
                    batch[k] = batch[k].to(device=cfg.device, non_blocking=True)
        model(batch)
        if i >= num_batches:
            break

    # Disable calibrators
    for name, module in model.named_modules():
        if isinstance(module, quant_nn.TensorQuantizer):
            if module._calibrator is not None:
                module.enable_quant()
                module.disable_calib()
            else:
                module.enable()
            
def compute_amax(model, **kwargs):
    # Load calib result
    for name, module in model.named_modules():
        if isinstance(module, quant_nn.TensorQuantizer):
            if module._calibrator is not None:
                if isinstance(module._calibrator, calib.MaxCalibrator):
                    module.load_calib_amax()
                else:
                    module.load_calib_amax(**kwargs)
#             print(F"{name:40}: {module}")
    model.cuda()

## Make Dataset and DataLoader

In [222]:
import os
import subprocess

import numpy as np
import skimage.io

from datasets.base import BaseDataset
from utils.boxes import generate_anchors
from PIL import Image
from torchvision.datasets.folder import default_loader


class LPR_CalibTRT(BaseDataset):
    def __init__(self, phase, cfg):
        super(LPR_CalibTRT, self).__init__(phase, cfg)

        self.input_size = (128, 128)  # (height, width), both dividable by 16
        self.class_names = ('0')
        # real_filtered mean and std
        # self.rgb_mean = np.array([94.87347, 96.89165, 94.70493], dtype=np.float32).reshape(1, 1, 3)
        # self.rgb_std = np.array([53.869507, 53.936283, 55.2807], dtype=np.float32).reshape(1, 1, 3)
        
        # real_filtered plus all_sites_seatbelt mean and std
        # self.rgb_mean = np.array([104.90631, 105.41336, 104.70162], dtype=np.float32).reshape(1, 1, 3)
        # self.rgb_std = np.array([50.69564, 49.60443, 50.158844], dtype=np.float32).reshape(1, 1, 3)

        self.rgb_mean = np.array([97.631615, 98.70732, 98.41285], dtype=np.float32).reshape(1, 1, 3)
        self.rgb_std = np.array([52.766678, 52.63513, 52.348827], dtype=np.float32).reshape(1, 1, 3)

        self.num_classes = len(self.class_names)
        self.class_ids_dict = {cls_name: cls_id for cls_id, cls_name in enumerate(self.class_names)}

        self.data_dir = os.path.join(cfg.data_dir, 'lpr_crop/merged_data')
        self.sample_ids, self.sample_set_path = self.get_sample_ids()
        self.grid_size = tuple(x // cfg.stride for x in self.input_size)  # anchors grid 
        # self.anchors_seed = np.array([[ 29, 17], [46, 32], [69, 52],
        #                                 [109, 68], [84, 127], [155, 106], 
        #                                 [255, 145], [183, 215], [371, 221]], dtype=np.float32) ## real_filtered anchors
        
        # self.anchors_seed = np.array( [[ 32, 20], [ 61, 42], [ 59, 97],
        #                                 [103, 66], [122, 114], [183, 96],
        #                                 [160, 152], [211, 201], [343, 205]], dtype=np.float32) ## real_filtered plus all_sites_seatbelt anchors

        # self.anchors_seed = np.array([[6, 5], [12, 10], [18, 10], [18, 18], [20, 24], [30, 15]], dtype=np.float32)
        self.anchors_seed = np.array([[3, 3], [6, 5], [9, 5], [10, 9], [10, 12], [15, 8]], dtype=np.float32)
        self.anchors = generate_anchors(self.grid_size, self.input_size, self.anchors_seed)
        self.anchors_per_grid = self.anchors_seed.shape[0]
        self.num_anchors = self.anchors.shape[0]

        self.results_dir = os.path.join(cfg.save_dir, 'results')

    def get_sample_ids(self):
        #a dirty duct tape to load preprocessing of val phase but load dataset for train phase for trt calib
        sample_set_name = 'train.txt' if self.phase == 'train' \
            else 'train.txt' if self.phase == 'val' \
            else 'trainval.txt' if self.phase == 'trainval' \
            else None

        sample_ids_path = os.path.join(self.data_dir, sample_set_name)
        with open(sample_ids_path, 'r') as fp:
            sample_ids = fp.readlines()
        sample_ids = tuple(x.strip() for x in sample_ids)

        return sample_ids, sample_ids_path

    def load_image(self, index):
        image_id = self.sample_ids[index]
        image_path = os.path.join(self.data_dir, 'images', image_id + '.png')
        image = default_loader(image_path)
        if image.mode == 'L':
            image = image.convert('RGB')
        image = np.array(image).astype(np.float32)
        # image = skimage.io.imread(image_path).astype(np.float32)
        return image, image_id

    def load_annotations(self, index):
        ann_id = self.sample_ids[index]
        ann_path = os.path.join(self.data_dir, 'labels', ann_id + '.txt')
        with open(ann_path, 'r') as fp:
            annotations = fp.readlines()

        annotations = [ann.strip().split(' ') for ann in annotations]
        class_ids, boxes = [], []
        for ann in annotations:
            if ann[0] not in self.class_names:
                continue
            class_ids.append(self.class_ids_dict[ann[0]])
            box = [float(x) for x in ann[4:8]]
            # if box[2] <= 0:
            #     box[2] = 0.00001
            # if box[3] <= 0:
            #     box[3] = 0.00001
            boxes.append(box)

        class_ids = np.array(class_ids, dtype=np.int16)
        boxes = np.array(boxes, dtype=np.float32)
        if len(boxes):
            return class_ids, boxes
        boxes = None
        return class_ids, boxes

    # ========================================
    #                evaluation
    # ========================================

    def save_results(self, results):
        txt_dir = os.path.join(self.results_dir, 'data')
        os.makedirs(txt_dir, exist_ok=True)

        for res in results:
            txt_path = os.path.join(txt_dir, res['image_meta']['image_id'] + '.txt')
            if 'class_ids' not in res:
                with open(txt_path, 'w') as fp:
                    fp.write('')
                continue

            num_boxes = len(res['class_ids'])
            with open(txt_path, 'w') as fp:
                for i in range(num_boxes):
                    class_name = self.class_names[res['class_ids'][i]].lower()
                    score = res['scores'][i]
                    bbox = res['boxes'][i, :]
                    line = '{} -1 -1 0 {:.2f} {:.2f} {:.2f} {:.2f} 0 0 0 0 0 0 0 {:.3f}\n'.format(
                            class_name, *bbox, score)
                    fp.write(line)

    def evaluate(self):
        kitti_eval_tool_path = os.path.join(self.cfg.root_dir, 'src/utils/kitti-eval/cpp/evaluate_object')
        cmd = '{} {} {} {} {}'.format(kitti_eval_tool_path,
                                      os.path.join(self.data_dir),
                                      self.sample_set_path,
                                      self.results_dir,
                                      len(self.sample_ids))

        status = subprocess.call(cmd, shell=True)

        aps = {}
        for class_name in self.class_names:
            map_path = os.path.join(self.results_dir, 'stats_{}_ap.txt'.format(class_name.lower()))
            if os.path.exists(map_path):
                with open(map_path, 'r') as f:
                    lines = f.readlines()
                _aps = [float(line.split('=')[1].strip()) for line in lines]
            else:
                _aps = [0., 0., 0.]

            aps[class_name + '_easy'] = _aps[0]
            aps[class_name + '_moderate'] = _aps[1]
            aps[class_name + '_hard'] = _aps[2]

        aps['mAP'] = sum(aps.values()) / len(aps)

        return aps



In [223]:
lpr_calib_dataset = LPR_CalibTRT('val', cfg)

In [224]:
calib_data_loader = torch.utils.data.DataLoader(lpr_calib_dataset,
                                                  batch_size=cfg.batch_size,
                                                  num_workers=cfg.num_workers,
                                                  pin_memory=False)

In [225]:
# print (len(calib_data_loader))
# inp  = next(iter(calib_data_loader))
# print(inp['image'].shape)

# for i, inp in enumerate(calib_data_loader, 0):
#         print(inp['image'].shape)
#         if i >= 10:
#             break

In [226]:
num_batches = int(len(lpr_calib_dataset)/cfg.batch_size)
print (num_batches)
# lpr_model = detector.model
# lpr_model.cuda()
# print (lpr_model)
detector.model.eval()
detector.model.cuda()
with torch.no_grad():
    collect_stats(detector.model, calib_data_loader, num_batches=2000)
    # compute_amax(detector.model, method="percentile", percentile=99.99)

I0811 22:36:31.199576 139664402845888 tensor_quantizer.py:183] Disable `quant` stage.
I0811 22:36:31.199994 139664402845888 tensor_quantizer.py:179] Enable HistogramCalibrator
I0811 22:36:31.200503 139664402845888 tensor_quantizer.py:183] Disable `quant` stage.
I0811 22:36:31.200871 139664402845888 tensor_quantizer.py:179] Enable MaxCalibrator
I0811 22:36:31.201338 139664402845888 tensor_quantizer.py:183] Disable `quant` stage.
I0811 22:36:31.201673 139664402845888 tensor_quantizer.py:179] Enable HistogramCalibrator
I0811 22:36:31.202006 139664402845888 tensor_quantizer.py:183] Disable `quant` stage.
I0811 22:36:31.202298 139664402845888 tensor_quantizer.py:179] Enable MaxCalibrator
I0811 22:36:31.202584 139664402845888 tensor_quantizer.py:183] Disable `quant` stage.
I0811 22:36:31.202862 139664402845888 tensor_quantizer.py:179] Enable HistogramCalibrator
I0811 22:36:31.203136 139664402845888 tensor_quantizer.py:183] Disable `quant` stage.
I0811 22:36:31.203391 139664402845888 tensor_q

14068


100%|██████████| 2000/2000 [04:45<00:00,  7.00it/s]
I0811 22:41:16.852417 139664402845888 tensor_quantizer.py:187] Enable `quant` stage.
W0811 22:41:16.852936 139664402845888 tensor_quantizer.py:173] Disable HistogramCalibrator
I0811 22:41:16.853235 139664402845888 tensor_quantizer.py:187] Enable `quant` stage.
W0811 22:41:16.853570 139664402845888 tensor_quantizer.py:173] Disable MaxCalibrator
I0811 22:41:16.853879 139664402845888 tensor_quantizer.py:187] Enable `quant` stage.
W0811 22:41:16.854173 139664402845888 tensor_quantizer.py:173] Disable HistogramCalibrator
I0811 22:41:16.854442 139664402845888 tensor_quantizer.py:187] Enable `quant` stage.
W0811 22:41:16.854707 139664402845888 tensor_quantizer.py:173] Disable MaxCalibrator
I0811 22:41:16.854984 139664402845888 tensor_quantizer.py:187] Enable `quant` stage.
W0811 22:41:16.855242 139664402845888 tensor_quantizer.py:173] Disable HistogramCalibrator
I0811 22:41:16.855503 139664402845888 tensor_quantizer.py:187] Enable `quant` st

In [227]:
print (detector.model)

SqueezeDetWithLoss(
  (base): SqueezeDetBase(
    (quant): QuantStub()
    (dequant): DeQuantStub()
    (features): Sequential(
      (0): ConvBNReLU(
        (0): QuantConv2d(
          3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False
          (_input_quantizer): TensorQuantizer(8bit fake per-tensor amax=dynamic calibrator=HistogramCalibrator scale=1.0 quant)
          (_weight_quantizer): TensorQuantizer(8bit fake axis=0 amax=dynamic calibrator=MaxCalibrator scale=1.0 quant)
        )
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): ConvBNReLU(
            (0): QuantConv2d(
              32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False
              (_input_quantizer): TensorQuantizer(8bit fake per-tensor amax=dynamic calibrator=HistogramCalibrator scale=1.0 quant)
              (_weight_qu

### Try Different Scehmes and Eval Model

In [228]:
from datasets.lpr import LPR

eval_dataset = LPR('val', cfg)
eval_dataset.phase

'val'

In [229]:
with torch.no_grad():
    for method in ["percentile", "mse", "entropy"]:
        percentile = 99.99
        print(F" ############  {method} calibration ######################")
        if(method == 'percentile'):
            compute_amax(detector.model, method=method, percentile = percentile)
        else:
            compute_amax(detector.model, method=method)
        update_exp_dir(cfg, f"trt_calib_lpd_{method}")
        results = detector.detect_dataset(eval_dataset, cfg)
        eval_dataset.save_results(results)
        aps = eval_dataset.evaluate()
        for k, v in aps.items():
                print('{:<20} {:.3f}'.format(k, v))
        torch.save(detector.model.state_dict(), os.path.join(cfg.save_dir, f"quant_lp_squeezedet_{method}.pth"))

W0811 22:45:37.243749 139664402845888 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0811 22:45:37.244675 139664402845888 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([32, 1, 1, 1]).
W0811 22:45:37.245321 139664402845888 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0811 22:45:37.245630 139664402845888 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([32, 1, 1, 1]).
W0811 22:45:37.246155 139664402845888 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0811 22:45:37.246483 139664402845888 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([16, 1, 1, 1]).
W0811 22:45:37.247032 139664402845888 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0811 22:45:37.247324 139664402845888 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([96, 1, 1, 1]).
W0811 22:45:37.247841 139664402845888 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Siz

 ############percentile calibration ######################
eval: [0/1530] | data 0.050s | net 0.015s
eval: [20/1530] | data 0.004s | net 0.019s
eval: [40/1530] | data 0.006s | net 0.021s
eval: [60/1530] | data 0.008s | net 0.027s
eval: [80/1530] | data 0.014s | net 0.040s
eval: [100/1530] | data 0.029s | net 0.075s
eval: [120/1530] | data 0.005s | net 0.019s
eval: [140/1530] | data 0.031s | net 0.080s
eval: [160/1530] | data 0.007s | net 0.022s
eval: [180/1530] | data 0.001s | net 0.011s
eval: [200/1530] | data 0.014s | net 0.040s
eval: [220/1530] | data 0.046s | net 0.133s
eval: [240/1530] | data 0.016s | net 0.046s
eval: [260/1530] | data 0.005s | net 0.019s
eval: [280/1530] | data 0.001s | net 0.011s
eval: [300/1530] | data 0.011s | net 0.034s
eval: [320/1530] | data 0.004s | net 0.018s
eval: [340/1530] | data 0.011s | net 0.033s
eval: [360/1530] | data 0.022s | net 0.069s
eval: [380/1530] | data 0.008s | net 0.026s
eval: [400/1530] | data 0.022s | net 0.057s
eval: [420/1530] | data

sh: 1: gnuplot: not found
sh: 1: gnuplot: not found
Error: /undefinedfilename in (car_detection.eps)
Operand stack:

Execution stack:
   %interp_exit   .runexec2   --nostringval--   --nostringval--   --nostringval--   2   %stopped_push   --nostringval--   --nostringval--   --nostringval--   false   1   %stopped_push
Dictionary stack:
   --dict:956/1684(ro)(G)--   --dict:0/20(G)--   --dict:77/200(L)--
Current allocation mode is local
Last OS error: No such file or directory
GPL Ghostscript 9.26: Unrecoverable error, exit code 1
sh: 1: pdfcrop: not found


Loading detections...
  done.
Your evaluation results are available in /home/gj/hazen/QAT_Squeezedet/SqueezeDet-PyTorch/exp/test_lpd_trt_calib/results
0_easy               0.892
0_moderate           0.892
0_hard               0.892
mAP                  0.892
 ############mse calibration ######################


W0811 22:46:40.742229 139664402845888 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0811 22:46:40.743023 139664402845888 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([32, 1, 1, 1]).
W0811 22:46:43.376795 139664402845888 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0811 22:46:43.377391 139664402845888 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([32, 1, 1, 1]).
W0811 22:46:44.646888 139664402845888 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0811 22:46:44.647568 139664402845888 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([16, 1, 1, 1]).
W0811 22:46:47.974412 139664402845888 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0811 22:46:47.975063 139664402845888 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([96, 1, 1, 1]).
W0811 22:46:49.985022 139664402845888 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Siz

...

W0811 22:48:31.195484 139664402845888 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0811 22:48:31.196293 139664402845888 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([32, 1, 1, 1]).
W0811 22:49:07.994331 139664402845888 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0811 22:49:08.011952 139664402845888 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([32, 1, 1, 1]).
W0811 22:49:17.569306 139664402845888 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0811 22:49:17.569837 139664402845888 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([16, 1, 1, 1]).
W0811 22:50:02.644994 139664402845888 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0811 22:50:02.651050 139664402845888 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([96, 1, 1, 1]).
W0811 22:50:20.069070 139664402845888 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Siz

eval: [0/1530] | data 0.003s | net 0.013s
eval: [20/1530] | data 0.004s | net 0.018s
eval: [40/1530] | data 0.006s | net 0.022s
eval: [60/1530] | data 0.008s | net 0.028s
eval: [80/1530] | data 0.014s | net 0.040s
eval: [100/1530] | data 0.029s | net 0.075s
eval: [120/1530] | data 0.004s | net 0.019s
eval: [140/1530] | data 0.031s | net 0.079s
eval: [160/1530] | data 0.007s | net 0.022s
eval: [180/1530] | data 0.011s | net 0.012s
eval: [200/1530] | data 0.013s | net 0.040s
eval: [220/1530] | data 0.046s | net 0.135s
eval: [240/1530] | data 0.017s | net 0.048s
eval: [260/1530] | data 0.015s | net 0.021s
eval: [280/1530] | data 0.001s | net 0.012s
eval: [300/1530] | data 0.014s | net 0.034s
eval: [320/1530] | data 0.004s | net 0.018s
eval: [340/1530] | data 0.014s | net 0.034s
eval: [360/1530] | data 0.022s | net 0.068s
eval: [380/1530] | data 0.008s | net 0.026s
eval: [400/1530] | data 0.023s | net 0.056s
eval: [420/1530] | data 0.003s | net 0.015s
eval: [440/1530] | data 0.037s | net 0

mkdir: cannot create directory ‘/home/gj/hazen/QAT_Squeezedet/SqueezeDet-PyTorch/exp/test_lpd_trt_calib/results/plot’: File exists
sh: 1: gnuplot: not found
sh: 1: gnuplot: not found


Loading detections...
  done.
Your evaluation results are available in /home/gj/hazen/QAT_Squeezedet/SqueezeDet-PyTorch/exp/test_lpd_trt_calib/results
0_easy               0.891
0_moderate           0.891
0_hard               0.891
mAP                  0.891


Error: /undefinedfilename in (car_detection.eps)
Operand stack:

Execution stack:
   %interp_exit   .runexec2   --nostringval--   --nostringval--   --nostringval--   2   %stopped_push   --nostringval--   --nostringval--   --nostringval--   false   1   %stopped_push
Dictionary stack:
   --dict:956/1684(ro)(G)--   --dict:0/20(G)--   --dict:77/200(L)--
Current allocation mode is local
Last OS error: No such file or directory
GPL Ghostscript 9.26: Unrecoverable error, exit code 1
sh: 1: pdfcrop: not found


'val'

In [ ]:
# # print (detector.model)
# print (detector.model.base.features[0][0].input_quantizer.amax)
# print (detector.model.base.features[0][0].weight_quantizer.amax)

In [ ]:
# eval_dataset.save_results(results)
# aps = eval_dataset.evaluate()
# for k, v in aps.items():
#         print('{:<20} {:.3f}'.format(k, v))

In [177]:
# torch.save(detector.model.state_dict(), "../exp/quant_lp_squeezedet.pth")

In [ ]:
# test = detector.model.base
# test.cpu()
# quant_nn.TensorQuantizer.use_fb_fake_quant = True
# dummy_input = torch.randn(1, 3, 128, 128)

# # input_names = [ "network.0" ]
# # output_names = [ "network.23" ]

# # enable_onnx_checker needs to be disabled. See notes below.
# torch.onnx.export(
#     test, dummy_input, "../exp/test.onnx", verbose=True, opset_version=13, enable_onnx_checker=False)